In [1]:
#Import dependencies
import requests
import pandas as pd
import re
import sqlite3
import uuid

# Astronauts Cleanup

In [2]:
#Read csv
file = 'astronauts.csv'
astro_df = pd.read_csv(file)
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
0,Joseph M. Acaba,2004.0,19.0,Active,5/17/1967,"Inglewood, CA",Male,University of California-Santa Barbara; Univer...,Geology,Geology,NaN,NaN,2,3307,2,13.0,"STS-119 (Discovery), ISS-31/32 (Soyuz)",NaN,NaN
1,Loren W. Acton,NaN,NaN,Retired,3/7/1936,"Lewiston, MT",Male,Montana State University; University of Colorado,Engineering Physics,Solar Physics,NaN,NaN,1,190,0,0.0,STS 51-F (Challenger),NaN,NaN
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987.0,12.0,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963.0,3.0,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN


In [3]:
#Remove all years greater than 1991
astro_df = astro_df[astro_df['Year'] <= 1991]
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984.0,10.0,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987.0,12.0,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963.0,3.0,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987.0,12.0,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967.0,6.0,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [4]:
#Change floats to int
astro_df["Year"] = astro_df["Year"].astype(int)
astro_df["Group"] = astro_df["Group"].astype(int)

astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [5]:
#Strip 'jr' from last name
def strip_jr(name):
    if "Jr." in name:
        result = name[:-4]
    else:
        result = name
    return (result)

astro_df['Name'] = astro_df['Name'].apply(strip_jr)
astro_df.head()

,Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN


In [6]:
#Split functions
def split_first(astro_string):
    if "." in astro_string:
        if "." in astro_string.split(".")[1]:
            first_name = astro_string.split('.')[0] + "." + astro_string.split('.')[0] + "."
        else:
            first_name = astro_string.split('.')[0] + "."
    else:
        first_name = astro_string.split()[0]
    return(first_name)
        
def split_last(astro_string):
    if "." in astro_string:
        last_name = astro_string.split('.')[1]
        last_name = last_name.strip()
    else:
        last_name = astro_string.split()[1]
    return(last_name)

#Assign to new columns
astro_df['First Name'] = astro_df['Name'].apply(split_first)
astro_df['Last Name'] = astro_df['Name'].apply(split_last)


#Rename Name to Full-Name
astro_df = astro_df.rename(columns={"Name":"Full Name"})
astro_df.head()

,Full Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,...,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Missions,Death Date,Death Mission,First Name,Last Name
2,James C. Adamson,1984,10,Retired,3/3/1946,"Warsaw, NY",Male,US Military Academy; Princeton University,Engineering,Aerospace Engineering,...,US Army (Retired),2,334,0,0.0,"STS-28 (Columbia), STS-43 (Atlantis)",NaN,NaN,James C.,Adamson
3,Thomas D. Akers,1987,12,Retired,5/20/1951,"St. Louis, MO",Male,University of Missouri-Rolla,Applied Mathematics,Applied Mathematics,...,US Air Force (Retired),4,814,4,29.0,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",NaN,NaN,Thomas D.,Akers
4,Buzz Aldrin,1963,3,Retired,1/20/1930,"Montclair, NJ",Male,US Military Academy; MIT,Mechanical Engineering,Astronautics,...,US Air Force (Retired),2,289,2,8.0,"Gemini 12, Apollo 11",NaN,NaN,Buzz,Aldrin
5,Andrew M. Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,...,US Marine Corps (Retired),3,906,0,0.0,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",NaN,NaN,Andrew M.,Allen
6,Joseph P. Allen,1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,DePauw University; Yale University,Mathematics & Physics,Physics,...,NaN,2,313,2,12.0,"ST-5 (Columbia), STS 51-A (Discovery)",NaN,NaN,Joseph P.,Allen


In [7]:
#Reorder Columns
f_column = astro_df.pop('First Name')
l_column = astro_df.pop('Last Name')
mission_column = astro_df.pop('Missions')
astro_df.insert(1, f_column.name, f_column)
astro_df.insert(2, l_column.name, l_column)
astro_df.insert(3, mission_column.name, mission_column)
astro_df.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
2,James C. Adamson,James C.,Adamson,"STS-28 (Columbia), STS-43 (Atlantis)",1984,10,Retired,3/3/1946,"Warsaw, NY",Male,...,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,NaN,NaN
3,Thomas D. Akers,Thomas D.,Akers,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",1987,12,Retired,5/20/1951,"St. Louis, MO",Male,...,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,NaN,NaN
4,Buzz Aldrin,Buzz,Aldrin,"Gemini 12, Apollo 11",1963,3,Retired,1/20/1930,"Montclair, NJ",Male,...,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,NaN,NaN
5,Andrew M. Allen,Andrew M.,Allen,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,...,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
6,Joseph P. Allen,Joseph P.,Allen,"ST-5 (Columbia), STS 51-A (Discovery)",1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,...,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,NaN,NaN


In [8]:
#Remove all astronauts who didn't pursue missions
astro_df = astro_df[astro_df['Missions'].notnull()]
astro_df.head()

,Full Name,First Name,Last Name,Missions,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
2,James C. Adamson,James C.,Adamson,"STS-28 (Columbia), STS-43 (Atlantis)",1984,10,Retired,3/3/1946,"Warsaw, NY",Male,...,Engineering,Aerospace Engineering,Colonel,US Army (Retired),2,334,0,0.0,NaN,NaN
3,Thomas D. Akers,Thomas D.,Akers,"STS-41 (Discovery), STS-49 (Endeavor), STS-61 ...",1987,12,Retired,5/20/1951,"St. Louis, MO",Male,...,Applied Mathematics,Applied Mathematics,Colonel,US Air Force (Retired),4,814,4,29.0,NaN,NaN
4,Buzz Aldrin,Buzz,Aldrin,"Gemini 12, Apollo 11",1963,3,Retired,1/20/1930,"Montclair, NJ",Male,...,Mechanical Engineering,Astronautics,Colonel,US Air Force (Retired),2,289,2,8.0,NaN,NaN
5,Andrew M. Allen,Andrew M.,Allen,"STS-46 (Atlantis), STS-62 (Columbia), STS-75 (...",1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,...,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
6,Joseph P. Allen,Joseph P.,Allen,"ST-5 (Columbia), STS 51-A (Discovery)",1967,6,Retired,6/27/1937,"Crawsfordsville, IN",Male,...,Mathematics & Physics,Physics,NaN,NaN,2,313,2,12.0,NaN,NaN


In [9]:
#Remove all mission related data
astro_df = astro_df.drop(["Missions"], axis=1)

#Apply sort based on Crew Member's Name
astro_df = astro_df.sort_values(by=["Full Name"])

#Reset index
astro_df = astro_df.reset_index(drop=True)

astro_df.head()

,Full Name,First Name,Last Name,Year,Group,Status,Birth Date,Birth Place,Gender,Alma Mater,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
0,Alan B. Shepard,Alan B.,Shepard,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,US Naval Academy,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
1,Alan L. Bean,Alan L.,Bean,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,University of Texas,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
2,Alfred M. Worden,Alfred M.,Worden,1966,5,Retired,2/7/1932,"Jackson, MI",Male,US Military Academy; University of Michigan,Military Science,Aeronautical & Astronautical Engineering,Colonel,US Air Force (Retired),1,295,1,0.5,NaN,NaN
3,Andrew M. Allen,Andrew M.,Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,Villanova University; University of Florida,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
4,Anna L. Fisher,Anna L.,Fisher,1978,8,Management,8/24/1949,"New York, NY",Female,University of California-Los Angeles,Chemistry,Chemistry; Medicine,NaN,NaN,1,191,0,0.0,NaN,NaN


In [10]:
#insert first column - astro_id 
def assign_astro_id(item):
    result = "A-"+str(uuid.uuid4().fields[-1])[:5]
    return(result)
astro_df["Astro ID"] = astro_df["Full Name"].apply(assign_astro_id)

id_column = astro_df.pop('Astro ID')
astro_df.insert(0, id_column.name, id_column)

astro_df

,Astro ID,Full Name,First Name,Last Name,Year,Group,Status,Birth Date,Birth Place,Gender,...,Undergraduate Major,Graduate Major,Military Rank,Military Branch,Space Flights,Space Flight (hr),Space Walks,Space Walks (hr),Death Date,Death Mission
0,A-21199,Alan B. Shepard,Alan B.,Shepard,1959,1,Deceased,11/18/1923,"East Derry, NH",Male,...,Naval Sciences,NaN,Rear Admiral,US Navy (Retired),2,216,2,9.0,7/21/1998,NaN
1,A-21584,Alan L. Bean,Alan L.,Bean,1963,3,Retired,3/15/1932,"Wheeler, TX",Male,...,Aeronautical Engineering,NaN,Captain,US Navy (Retired),2,1671,3,10.0,NaN,NaN
2,A-58143,Alfred M. Worden,Alfred M.,Worden,1966,5,Retired,2/7/1932,"Jackson, MI",Male,...,Military Science,Aeronautical & Astronautical Engineering,Colonel,US Air Force (Retired),1,295,1,0.5,NaN,NaN
3,A-83466,Andrew M. Allen,Andrew M.,Allen,1987,12,Retired,8/4/1955,"Philadelphia, PA",Male,...,Mechanical Engineering,Business Administration,Lieutenant Colonel,US Marine Corps (Retired),3,906,0,0.0,NaN,NaN
4,A-10163,Anna L. Fisher,Anna L.,Fisher,1978,8,Management,8/24/1949,"New York, NY",Female,...,Chemistry,Chemistry; Medicine,NaN,NaN,1,191,0,0.0,NaN,NaN
5,A-44933,Anthony W. England,Anthony W.,England,1967,6,Retired,5/15/1942,"Indianapolis, IN",Male,...,Geology,Geology; Geophysics,NaN,NaN,1,190,0,0.0,NaN,NaN
6,A-14344,Bernard A. Harris,Bernard A.,Harris,1990,13,Retired,6/26/1956,"Temple, TX",Male,...,Biology,Medicine,NaN,NaN,2,438,1,5.0,NaN,NaN
7,A-13120,Bonnie J. Dunbar,Bonnie J.,Dunbar,1980,9,Retired,3/3/1949,"Sunnyside, WA",Female,...,Ceramic Engineering,Ceramic Engineering; Biomedical Engineering,NaN,NaN,5,1207,0,0.0,NaN,NaN
8,A-13074,Brewster H. Shaw,Brewster H.,Shaw,1978,8,Retired,5/16/1945,"Cass City, MI",Male,...,Engineering Mechanics,Engineering Mechanics,Colonel,US Air Force (Retired),3,533,0,0.0,NaN,NaN
9,A-24473,Brian Duffy,Brian,Duffy,1985,11,Retired,6/20/1953,"Boston, MA",Male,...,Mathematics,Systems Management,Colonel,US Air Force (Retired),4,977,0,0.0,NaN,NaN


# Mission Cleanup

In [11]:
#Set up url
url = 'https://www.windows2universe.org/space_missions/manned_table.html'

#Retrieve page
tables = pd.read_html(url)

#Create df
df_mission = tables[0]

df_mission.head()

,0,1,2,3,4
0,Mission(Country),Craft,Launch Date,Crew,Mission Highlights
1,Vostok-1(USSR),Kedr (Cedar),"April 12, 1961",Gagarin,Cosmonaut Yuri Garagin became the first human ...
2,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
3,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
4,Vostok-2(USSR),Orel (Eagle),"August 6, 1961",Titov,Titov was the first to spend an entire day in ...


In [12]:
#Set up headers
df_mission.columns = df_mission.iloc[0]
df_mission = df_mission.drop(0)

#Delete "(Country)" from first column header
df_mission = df_mission.rename(columns={"Mission(Country)":"Mission"})

#Delete all rows with "(USSR)" in Mission name
df_mission = df_mission[~df_mission['Mission'].str.contains('(USSR)')]

df_mission.head()

C:\Users\Asha\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,Mission,Craft,Launch Date,Crew,Mission Highlights
2,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...
3,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ..."
5,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...
6,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...
9,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...


In [13]:
# Rename Craft field to more accurately describe ship name
df_mission.rename(columns={"Craft":"Given Craft Name"},inplace=True)

In [14]:
#Split spacecraft model from Mission name 
def split_craft(craft_string):
#     if "-" in craft_string:
#         craft_name = craft_string.split('-')[0]
#     else:
    craft_name = craft_string.split()[0]
    return(craft_name)
        
# def split_last(astro_string):
#     if "." in astro_string:
#         last_name = astro_string.split('.')[1]
#         last_name = last_name.strip()
#     else:
#         last_name = astro_string.split()[1]
#     return(last_name)

#Assign to new columns
df_mission['Craft Model'] = df_mission['Given Craft Name'].apply(split_craft)
df_mission

,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
2,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
3,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
5,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
6,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
9,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma
10,Mercury 9(USA),Faith 7,"May 15, 1963",Cooper,Cooper pilots the longest and last Mercury mis...,Faith
15,Gemini-Titan 3(USA),Molly Brown,"March 23, 1965",GrissomYoung,"Starting the Gemini program, this spacecraft w...",Molly
16,Gemini-Titan 4(USA),Gemini 4,"June 3, 1965",McDivittWhite,This mission set a four-day endurance record. ...,Gemini
17,Gemini-Titan 5(USA),Gemini 5,"August 21, 1965",Cooper Conrad,Cooper and Conrad set the world endurance reco...,Gemini
18,Gemini-Titan 7(USA),Gemini 7,"December 4, 1965",Borman Lovell,This mission set yet another endurance record ...,Gemini


In [15]:
#Mission id assigning
def assign_mission_id(item):
    result = "M-"+str(uuid.uuid4().fields[-1])[:5]
    return(result)
df_mission["Mission ID"] = df_mission["Mission"].apply(assign_mission_id)

id_column = df_mission.pop('Mission ID')
df_mission.insert(0, id_column.name, id_column)
df_mission

,Mission ID,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
2,M-14469,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
3,M-10093,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
5,M-77477,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
6,M-22015,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
9,M-23717,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma
10,M-26425,Mercury 9(USA),Faith 7,"May 15, 1963",Cooper,Cooper pilots the longest and last Mercury mis...,Faith
15,M-14189,Gemini-Titan 3(USA),Molly Brown,"March 23, 1965",GrissomYoung,"Starting the Gemini program, this spacecraft w...",Molly
16,M-79354,Gemini-Titan 4(USA),Gemini 4,"June 3, 1965",McDivittWhite,This mission set a four-day endurance record. ...,Gemini
17,M-13645,Gemini-Titan 5(USA),Gemini 5,"August 21, 1965",Cooper Conrad,Cooper and Conrad set the world endurance reco...,Gemini
18,M-23497,Gemini-Titan 7(USA),Gemini 7,"December 4, 1965",Borman Lovell,This mission set yet another endurance record ...,Gemini


In [16]:
#Reset Index
df_mission = df_mission.reset_index(drop=True)
df_mission.head()

,Mission ID,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
0,M-14469,Mercury-3(USA),Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4(USA),Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6(USA),Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7(USA),Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8(USA),Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma


In [17]:
#Delete country naming
def remove_usa(name):
    result = name.split('(')[0]
    return(result)

df_mission['Mission'] = df_mission["Mission"].apply(remove_usa)
df_mission.head()

,Mission ID,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
0,M-14469,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma


In [18]:
#Splitting df for id and joining purposes
join_df = df_mission

mission_df=df_mission.drop(["Crew"], axis=1)
mission_df.head()

,Mission ID,Mission,Given Craft Name,Launch Date,Mission Highlights,Craft Model
0,M-14469,Mercury-3,Freedom 7,"May 5, 1961",Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4,Liberty Bell-7,"July 21, 1961","The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6,Friendship 7,"February 20,1962",The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7,Aurora 7,"May 24, 1962",The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8,Sigma 7,"October 3, 1962",Walter Schirra orbited the Earth six times dur...,Sigma


In [19]:
#Remove leading characters and commas from crew column (currently they are only leading, not deliminating)
join_df['Crew'] = join_df['Crew'].str.strip(',')
join_df.head()

,Mission ID,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
0,M-14469,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma


In [20]:
#Split names for sorting
def split_it(crew_string):
    result = re.sub(r"(\w)([A-Z])", r"\1 \2", crew_string)

    if 'Mac' in result:
        result = result.replace("Mac ", "Mac")
    
    if "Mc" in result:
        result = result.replace("Mc ", "Mc")
        
    if "van den Berg" in result:
        result = "vandenBerg"
    
    if "van Hoften" in result:
        result = "vanHoften"

    return(result)

join_df['Crew'] = join_df['Crew'].apply(split_it)
join_df.head()

,Mission ID,Mission,Given Craft Name,Launch Date,Crew,Mission Highlights,Craft Model
0,M-14469,Mercury-3,Freedom 7,"May 5, 1961",Shepard,Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4,Liberty Bell-7,"July 21, 1961",Grissom,"The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6,Friendship 7,"February 20,1962",Glenn,The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7,Aurora 7,"May 24, 1962",Carpenter,The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8,Sigma 7,"October 3, 1962",Schirra,Walter Schirra orbited the Earth six times dur...,Sigma


In [21]:
#Move Crew column to second column
my_column = join_df.pop('Crew')
join_df.insert(2, my_column.name, my_column)
join_df.head()

,Mission ID,Mission,Crew,Given Craft Name,Launch Date,Mission Highlights,Craft Model
0,M-14469,Mercury-3,Shepard,Freedom 7,"May 5, 1961",Alan Shepard became the first American in spac...,Freedom
1,M-10093,Mercury-4,Grissom,Liberty Bell-7,"July 21, 1961","The second US sub-orbital flight, reaching an ...",Liberty
2,M-77477,Mercury-6,Glenn,Friendship 7,"February 20,1962",The first US manned orbital flight at an orbit...,Friendship
3,M-22015,Mercury-7,Carpenter,Aurora 7,"May 24, 1962",The second US manned orbital flight which orbi...,Aurora
4,M-23717,Mercury-8,Schirra,Sigma 7,"October 3, 1962",Walter Schirra orbited the Earth six times dur...,Sigma


In [22]:
#Setting up name split per mission
df_join = pd.DataFrame(columns=join_df.columns)
df_join

#Iterate over df in order to create new df, split names appropriately so each crew member is seperate
for index, row in join_df.iterrows():
    check = row["Crew"].split(" ")
    
    for name in check:
        missionid = row["Mission ID"]
        mission = row["Mission"]
        craft = row["Given Craft Name"]
        launchdate= row["Launch Date"]
        missionhighlights = row["Mission Highlights"]
        model = row["Craft Model"]
        simp_df = pd.DataFrame([[missionid, mission, name, craft, launchdate, missionhighlights,model]], columns=df_join.columns)
        df_join = df_join.append(simp_df)

#Apply sort based on Crew Member's Last Name
df_join = df_join.sort_values(by=["Mission"])

#Reset index
df_join = df_join.reset_index(drop=True)

df_join.head()

,Mission ID,Mission,Crew,Given Craft Name,Launch Date,Mission Highlights,Craft Model
0,M-10478,41-B,Brand,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
1,M-10478,41-B,Gibson,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
2,M-10478,41-B,McNair,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
3,M-10478,41-B,Stewart,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
4,M-10478,41-B,McCandless,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger


In [23]:
#Final Name fix for the "van" leaders
def fix_name(crew_name):
    if "vandenBerg" in crew_name:
        name = "van den Berg"  
    elif "vanHoften" in crew_name:
        name = "van Hoften"
    else:
        name = crew_name
        
    return(name)

df_join['Crew'] = df_join['Crew'].apply(fix_name)
df_join

,Mission ID,Mission,Crew,Given Craft Name,Launch Date,Mission Highlights,Craft Model
0,M-10478,41-B,Brand,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
1,M-10478,41-B,Gibson,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
2,M-10478,41-B,McNair,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
3,M-10478,41-B,Stewart,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
4,M-10478,41-B,McCandless,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger
5,M-85349,41-C,van Hoften,Challenger,"April 6, 1984","This mission accomplished the first capture, r...",Challenger
6,M-48283,41-D,Mullane,Discovery,"August 30, 1984",First flight of the Shuttle Discovery. The Con...,Discovery
7,M-48283,41-D,Hawley,Discovery,"August 30, 1984",First flight of the Shuttle Discovery. The Con...,Discovery
8,M-48283,41-D,Coats,Discovery,"August 30, 1984",First flight of the Shuttle Discovery. The Con...,Discovery
9,M-48283,41-D,Hartsfield,Discovery,"August 30, 1984",First flight of the Shuttle Discovery. The Con...,Discovery


# Spaceship Cleanup

In [24]:
# Used Pandas to retrieve data from table in html rather than BeautifulSoup
# First table of Orbital crewed spaceship
df1 = (pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_crewed_space_vehicles'))[1]
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*
1,SpaceShipOne,USA,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18]
2,X-15,USA,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19]
3,New Shepard,USA,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19]
4,SpaceShipTwo,USA,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21]


In [25]:
# Gets first row as a list to use as new header
header_1 = df1.iloc[0]
header_1

0             Spacecraft
1                 Origin
2           Manufacturer
3                  Range
4           Launchsystem
5               Crewsize
6             Length (m)
7           Diameter (m)
8       Launch mass (kg)
9            Powersystem
10    Generatedpower (W)
11     Firstspaceflight*
12       Lastspaceflight
13              Flights*
Name: 0, dtype: object

In [26]:
#Rename columns for proper labels
df1.rename(columns=header_1, inplace=True)
df1.head()

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*
1,SpaceShipOne,USA,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18]
2,X-15,USA,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19]
3,New Shepard,USA,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19]
4,SpaceShipTwo,USA,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21]


In [27]:
# Add orbital type that separates table subjects
df1['Orbit'] = "Orbital"

In [28]:
# Second table of Sub-orbital crewed spaceship
df2 = (pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_crewed_space_vehicles'))[0]
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]


In [29]:
#Set Column names
df2 = df2.rename(columns=df2.iloc[0]).drop(df2.index[0])
df2.head()

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Payloadto/from ISS (kg),Firstspaceflight*,Lastspaceflight,Flights*
1,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1]
2,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3]
3,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3]
4,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,"12,500/16,000",1981,2011,135[note 6]
5,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7]


In [30]:
#Rename Orbit type
df2['Orbit'] = "Sub-Orbital"

In [31]:
# See which columns can be joined on
list(df1.columns)

['Spacecraft',
 'Origin',
 'Manufacturer',
 'Range',
 'Launchsystem',
 'Crewsize',
 'Length (m)',
 'Diameter (m)',
 'Launch mass (kg)',
 'Powersystem',
 'Generatedpower (W)',
 'Firstspaceflight*',
 'Lastspaceflight',
 'Flights*',
 'Orbit']

In [32]:
# See which columns can be joined on
list(df2.columns)

['Spacecraft',
 'Origin',
 'Manufacturer',
 'Range',
 'Launchsystem',
 'Crewsize',
 'Length (m)',
 'Diameter (m)',
 'Launch mass (kg)',
 'Powersystem',
 'Payloadto/from ISS (kg)',
 'Firstspaceflight*',
 'Lastspaceflight',
 'Flights*',
 'Orbit']

In [33]:
#DF Merge
df3 = df1.merge(df2,how='outer')
df3

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*,Orbit,Payloadto/from ISS (kg)
0,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*,Orbital,NaN
1,SpaceShipOne,USA,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18],Orbital,NaN
2,X-15,USA,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19],Orbital,NaN
3,New Shepard,USA,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19],Orbital,NaN
4,SpaceShipTwo,USA,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21],Orbital,NaN
5,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Sub-Orbital,NaN
6,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Sub-Orbital,NaN
7,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Sub-Orbital,NaN
8,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,NaN,1981,2011,135[note 6],Sub-Orbital,"12,500/16,000"
9,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Sub-Orbital,NaN


In [34]:
# Extract only US ships
ship_df = df3[df3.Origin == 'USA']
ship_df

,Spacecraft,Origin,Manufacturer,Range,Launchsystem,Crewsize,Length (m),Diameter (m),Launch mass (kg),Powersystem,Generatedpower (W),Firstspaceflight*,Lastspaceflight,Flights*,Orbit,Payloadto/from ISS (kg)
1,SpaceShipOne,USA,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18],Orbital,NaN
2,X-15,USA,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19],Orbital,NaN
3,New Shepard,USA,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19],Orbital,NaN
4,SpaceShipTwo,USA,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21],Orbital,NaN
5,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Sub-Orbital,NaN
6,Gemini,USA,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Sub-Orbital,NaN
7,Apollo,USA,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Sub-Orbital,NaN
8,Space Shuttle orbiter,USA,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,NaN,1981,2011,135[note 6],Sub-Orbital,"12,500/16,000"
9,Orion,USA,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Sub-Orbital,NaN
27,Biconic Space Vehicle,USA,Blue Origin,LEO,New Glenn,NaN,NaN,NaN,NaN,NaN,NaN,Planned date not known,NaN,0,Sub-Orbital,NaN


In [35]:
# Rename Launchsytem, Crewsize, Powersystem, Payloadto/from ISS (kg), Firstspaceflight*, Lastspaceflight, Flights* fields
ship_df = ship_df.rename(columns={"Launchsystem":"Launch System", "Crewsize":"Crew Size", "Powersystem":"Power System", "Payloadto/from ISS (kg)":"Payload to/from ISS (kg)", "Firstspaceflight*":"First Space Flight","Lastspaceflight":"Last Space Flight", "Flights*":"Flights"})
ship_df.head()

,Spacecraft,Origin,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Generatedpower (W),First Space Flight,Last Space Flight,Flights,Orbit,Payload to/from ISS (kg)
1,SpaceShipOne,USA,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18],Orbital,NaN
2,X-15,USA,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19],Orbital,NaN
3,New Shepard,USA,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19],Orbital,NaN
4,SpaceShipTwo,USA,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21],Orbital,NaN
5,Mercury,USA,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Sub-Orbital,NaN


In [36]:
#Remove origin
ship_df = ship_df.drop(["Origin"], axis=1)

In [37]:
# Reset index
ship_df.reset_index(drop=True,inplace=True)
ship_df

,Spacecraft,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Generatedpower (W),First Space Flight,Last Space Flight,Flights,Orbit,Payload to/from ISS (kg)
0,SpaceShipOne,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3[note 18],Orbital,NaN
1,X-15,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2[note 19],Orbital,NaN
2,New Shepard,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019(2015),NaN,0 (10)[note 19],Orbital,NaN
3,SpaceShipTwo,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8[note 20],18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2[note 21],Orbital,NaN
4,Mercury,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961 (1960),1963,6(12)[note 1],Sub-Orbital,NaN
5,Gemini,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC[note 2],2,5.56,3.05,NaN,Fuel cells,NaN,1965 (1964),1966,10(2)[note 3],Sub-Orbital,NaN
6,Apollo,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967 (1966),1975,15(4)[note 3],Sub-Orbital,NaN
7,Space Shuttle orbiter,United Space Alliance,LEO,Space Shuttle,8[note 4],37.24,4.8[note 5],109000,Fuel cells,NaN,1981,2011,135[note 6],Sub-Orbital,"12,500/16,000"
8,Orion,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021 (2014),NaN,0 (1)[note 7],Sub-Orbital,NaN
9,Biconic Space Vehicle,Blue Origin,LEO,New Glenn,NaN,NaN,NaN,NaN,NaN,NaN,Planned date not known,NaN,0,Sub-Orbital,NaN


In [38]:
# Remove square andd round bracketed notes from dataset
ship_df.replace("\[.*\]", "",regex=True, inplace=True)
ship_df.replace("\(.*\)", "",regex=True, inplace=True)
ship_df.head()

,Spacecraft,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Generatedpower (W),First Space Flight,Last Space Flight,Flights,Orbit,Payload to/from ISS (kg)
0,SpaceShipOne,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3,Orbital,NaN
1,X-15,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2,Orbital,NaN
2,New Shepard,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019,NaN,0,Orbital,NaN
3,SpaceShipTwo,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8,18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2,Orbital,NaN
4,Mercury,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961,1963,6,Sub-Orbital,NaN


In [39]:
def assign_ship_id(item):
    result = "S-"+str(uuid.uuid4().fields[-1])[:5]
    return(result)
ship_df["Spacecraft ID"] = ship_df["Spacecraft"].apply(assign_ship_id)

id_column = ship_df.pop('Spacecraft ID')
ship_df.insert(0, id_column.name, id_column)
ship_df

,Spacecraft ID,Spacecraft,Manufacturer,Range,Launch System,Crew Size,Length (m),Diameter (m),Launch mass (kg),Power System,Generatedpower (W),First Space Flight,Last Space Flight,Flights,Orbit,Payload to/from ISS (kg)
0,S-11453,SpaceShipOne,Scaled Composites,112 kmX Prize,White KnightHybrid Motor,1,8.53,8.05,3600,NaN,NaN,2004,2004,3,Orbital,NaN
1,S-15283,X-15,North American Aviation,108 kmaltitude,B-52Ammonia-LOX,1,15.45,6.8,15420,NaN,NaN,1963,1963,2,Orbital,NaN
2,S-18866,New Shepard,Blue Origin,103 km,New Shepard BE-3,6,NaN,NaN,NaN,NaN,NaN,Planned: 2019,NaN,0,Orbital,NaN
3,S-19059,SpaceShipTwo,The Spaceship Company,110 kmKármán line+10,White Knight TwoRocketMotorTwo,8,18.3,8.3,9740,NaN,NaN,December 13 2018,NaN,2,Orbital,NaN
4,S-13341,Mercury,McDonnell AircraftNorth American Aviation,LEOattained,Redstone MRLVAtlas LV-3B,1,3.34,1.89,NaN,Batteries,NaN,1961,1963,6,Sub-Orbital,NaN
5,S-18775,Gemini,McDonnell AircraftMartin,LEO,Titan II GLVTitan IIIC,2,5.56,3.05,NaN,Fuel cells,NaN,1965,1966,10,Sub-Orbital,NaN
6,S-35278,Apollo,North American AviationGrumman and Douglas,Lunar,Saturn IBSaturn V,3,8.5,3.91,"5,500 CM + 14,700 LM24,500 Service Module",Fuel cells,NaN,1967,1975,15,Sub-Orbital,NaN
7,S-20033,Space Shuttle orbiter,United Space Alliance,LEO,Space Shuttle,8,37.24,4.8,109000,Fuel cells,NaN,1981,2011,135,Sub-Orbital,"12,500/16,000"
8,S-26871,Orion,Lockheed MartinAstrium,Mars,Delta IV HeavySpace Launch System,6,3.3,5,"8,900 Capsule12,300 Service Module",Solar Panels,NaN,Planned: 2021,NaN,0,Sub-Orbital,NaN
9,S-12884,Biconic Space Vehicle,Blue Origin,LEO,New Glenn,NaN,NaN,NaN,NaN,NaN,NaN,Planned date not known,NaN,0,Sub-Orbital,NaN


# Joining and Merging

In [40]:
# Extract Astronaut fields to append to Join table
name1 = astro_df[["Astro ID", "Last Name"]]
name1 = name1.rename(columns={"Last Name":"Crew"})

df_join = pd.merge(df_join, name1, on="Crew")
df_join.drop(["Crew"], axis=1, inplace=True)
df_join


,Mission ID,Mission,Given Craft Name,Launch Date,Mission Highlights,Craft Model,Astro ID
0,M-10478,41-B,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger,A-11762
1,M-24776,STS-35,Columbia,"December 2, 1990",This Spacelab mission performed astronomical e...,Columbia,A-11762
2,M-25768,STS-5,Columbia,"November 11, 1982",First operational flight of the Space Shuttle ...,Columbia,A-11762
3,M-10478,41-B,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger,A-10849
4,M-10478,41-B,Challenger,"February 3, 1984",The first untethered spacewalk in history was ...,Challenger,A-39928
5,M-17792,61-C,Columbia,"January 12, 1986",This mission had lots of problems and had to b...,Columbia,A-10849
6,M-17792,61-C,Columbia,"January 12, 1986",This mission had lots of problems and had to b...,Columbia,A-39928
7,M-82816,STS-27,Atlantis,"December 2, 1988",The third classified Department of Defense Shu...,Atlantis,A-10849
8,M-82816,STS-27,Atlantis,"December 2, 1988",The third classified Department of Defense Shu...,Atlantis,A-39928
9,M-21096,Skylab SL-4,Skylab,"November 15, 1973",These astronauts became the final crew of Skyl...,Skylab,A-10849


In [41]:
# Extract Ship fields to append to Join table
ship1 = ship_df[["Spacecraft ID", "Spacecraft"]]
ship1 = ship1.rename(columns={"Spacecraft":"Craft Model"})

df_join = pd.merge(df_join, ship1, on="Craft Model")
df_join.drop(["Craft Model"], axis=1, inplace=True)
df_join


,Mission ID,Mission,Given Craft Name,Launch Date,Mission Highlights,Astro ID,Spacecraft ID
0,M-10205,Apollo-Saturn 10,Apollo 10cm: Charlie Brownlm: Snoopy,"May 18, 1969",This mission was a dress rehearsal for a lunar...,A-80226,S-35278
1,M-11991,Apollo-Saturn 17,Apollo 17cm: Challengerlm: America,"December 7, 1972",Apollo 17 was the last lunar landing mission. ...,A-80226,S-35278
2,M-10205,Apollo-Saturn 10,Apollo 10cm: Charlie Brownlm: Snoopy,"May 18, 1969",This mission was a dress rehearsal for a lunar...,A-25269,S-35278
3,M-21659,Apollo-Saturn 16,Apollo 16cm: Casperlm: Orion,"April 16, 1972",Young and Duke visited the previously unexplor...,A-25269,S-35278
4,M-10205,Apollo-Saturn 10,Apollo 10cm: Charlie Brownlm: Snoopy,"May 18, 1969",This mission was a dress rehearsal for a lunar...,A-70375,S-35278
5,M-39898,Apollo-Saturn 11,Apollo 11cm: Columbialm: Eagle,"July 16, 1969",Apollo 11 successfully completed the first man...,A-15342,S-35278
6,M-39898,Apollo-Saturn 11,Apollo 11cm: Columbialm: Eagle,"July 16, 1969",Apollo 11 successfully completed the first man...,A-16720,S-35278
7,M-39898,Apollo-Saturn 11,Apollo 11cm: Columbialm: Eagle,"July 16, 1969",Apollo 11 successfully completed the first man...,A-19345,S-35278
8,M-39898,Apollo-Saturn 11,Apollo 11cm: Columbialm: Eagle,"July 16, 1969",Apollo 11 successfully completed the first man...,A-37453,S-35278
9,M-24600,Apollo-Saturn 12,Apollo 12cm: Yankee Clipperlm: Intrepid,"November 14, 1969",Apollo 12 made the second landing on the moon....,A-13511,S-35278


# Load to SQL

In [42]:
#Build connection
conn = sqlite3.connect('NASA.db')  
c = conn.cursor()

In [43]:
#Load astro_df to table
astro_df.to_sql('ASTRONAUTS', conn, if_exists='replace', index=False)

#Load mission_df to table
mission_df.to_sql('MISSIONS', conn, if_exists='replace', index=False)

#Load ship_df to table
ship_df.to_sql('SPACESHIPS', conn, if_exists='replace', index=False)

#Loat join_df to table
df_join.to_sql('MASTER', conn, if_exists='replace', index=False)

C:\Users\Asha\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)
